In [1]:
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import pandas as pd
import re

# 正则提取格式
# 队名
findPN = re.compile(r'<p class="c-color-link c-line-clamp1" data-a-7ecb45f1="">(.*?)</p>')
# 数据
finddata = re.compile(r'<div class="score c-color-link" data-a-7ecb45f1="">(.*?)</div>')
# 选手排行榜名
rln = ['出场次数', 'KDA', '参团率', '场均击杀', '总击杀', '场均死亡', '总死亡', '场均助攻', '总助攻', 'GPM', '经济占比', '每分钟输出', '输出占比', '伤害转化率',
       '每分钟承伤', '承伤占比', '每分钟补刀', '英雄使用数']
datalist = []


def getData(url):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'
    }
    for i in range(0, 18):  # 循环获取网页
        url1 = url + str(i)
        request = urllib.request.Request(url1, headers=headers)
        response = urllib.request.urlopen(request)
        html = response.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")

        # 提取出每一个排行榜
        item = soup.find_all('div', class_="match-player-group")
        item = str(item)

        # 选手名
        pn = re.findall(findPN, item)

        # 数据
        data = re.findall(finddata, item)

        linkdata(pn, data)


def linkdata(pn, data):
    datadict = dict(zip(pn, data))
    datalist.append(datadict)


def todataframe():
    alldata = dict(zip(rln, datalist))
    df = pd.DataFrame(alldata)
    writer = pd.ExcelWriter('../data.xlsx', mode='a', engine='openpyxl')
    df.to_excel(writer, sheet_name='player data')
    writer.save()


if __name__ == '__main__':
    # 网页URL
    url = "https://tiyu.baidu.com/match/lpl/tab/%E9%80%89%E6%89%8B%E6%A6%9C/current/"
    # 获取数据
    getData(url)
    todataframe()
